# Representing text

If we want to solve Natural Language Processing (NLP) tasks with neural networks, we need some way to represent text as tensors. Computers already represent textual characters as numbers that map to fonts on your screen using encodings such as ASCII or UTF-8.

![Image showing diagram mapping a character to an ASCII and binary representation](./images/ascii-character-map.png)

We understand what each letter **represents**, and how all characters come together to form the words of a sentence. However, computers by themselves do not have such an understanding, and neural network has to learn the meaning during training.

Therefore, we can use different approaches when representing text:
* **Character-level representation**, when we represent text by treating each character as a number. Given that we have $C$ different characters in our text corpus, the word *Hello* would be represented by $5\times C$ tensor. Each letter would correspond to a tensor column in one-hot encoding.
* **Word-level representation**, in which we create a **vocabulary** of all words in our text, and then represent words using one-hot encoding. This approach is somehow better, because each letter by itself does not have much meaning, and thus by using higher-level semantic concepts - words - we simplify the task for the neural network. However, given large dictionary size, we need to deal with high-dimensional sparse tensors.

Let's start by installing some required Python packages we'll use in this module.

In [16]:
import sys
!{sys.executable} -m pip install --user --quiet tensorflow_datasets

Error processing line 3 of /anaconda/envs/py38_tensorflow/lib/python3.8/site-packages/googleapis_common_protos-1.53.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "/anaconda/envs/py38_tensorflow/lib/python3.8/site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 553, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


# Text classification task

In this module, we will start with a simple text classification task based on **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** dataset, which is to classify news headlines into one of 4 categories: World, Sports, Business and Sci/Tech. To load the dataset, we will use **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API.

> In the sandbox environment, we need to pre-fetch the dataset from a known location before creating it with TensorFlow datasets. If you are running in the local environment, you can skip the next cell, and TensorFlow Datasets library will download the data automatically.

In [17]:
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [18]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

dataset = tfds.load('ag_news_subset')

We can now access the training and test portions of the dataset by using `dataset['train']` and `dataset['test']` respectively:

In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

So, let's print out the first 10 new headlines from our dataset: 

In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Text Vectorization

Now we need to convert text into **numbers** that can be represented as tensors. If we want word-level representation, we need to do two things:

* use **tokenizer** to split text into **tokens**
* build a **vocabulary** of those tokens.

Both of those steps can be handled using **TextVectorization** layer. Let's first create this layer explicitly to see how it works. We first instantite the vectorizer object, and then call `adapt` method to go through all text and build a vocabulary:

In [13]:
vectorizer = keras.layers.experimental.preprocessing.TextVectorization()
vectorizer.adapt(ds_train.map(lambda x: x['title']+' '+x['description']))

Now we can access the actual vocabulary:

In [18]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'of', 'in', 'and', 'on', 'for']
Length of vocabulary: 102171


Using tokenizer, we can easily encode any text into a set of numbers:

In [6]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 372, 2297,    3,  312,   12, 1293, 2314])>

## Bag of Words text representation

Because words represent meaning, sometimes we can figure out the meaning of a text by just looking at the individual words, regardless of their order in the sentence. For example, when classifying news, words like *weather*, *snow* are likely to indicate *weather forecast*, while words like *stocks*, *dollar* would count towards *financial news*.

**Bag of Words** (BoW) vector representation is the most simple to understand traditional vector representation. Each word is linked to a vector index, vector element contains the number of occurrences of a word in a given document.

![Image showing how a bag of words vector representation is represented in memory.](./images/bag-of-words-example.png) 

> **Note**: You can also think of BoW as a sum of all one-hot-encoded vectors for individual words in the text.

Below is an example of how to generate a bag of word representation using the Scikit Learn python library:

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()



array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

We can also use Keras vectorizer that we have defined above, converting each word number into one-hot encoding and adding all those vectors up:

In [67]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

> **Note**: You may be surprised why the result differs from the previous example. The reason is that the length of the vector corresponds to vocabulary size, and in the Scikit Learn example we have build the vocabulary from the sample text on the fly. In the latter example, we were using the large vocabulary from the whole AG News dataset.

## Limiting Vocabulary Size

In our case, the vocabulary size was rather big, more than 100k words. Sometimes we do not need words that are rarely present in the text, because they likely will not play an important role in machine learning tasks, because only few sentences will have them, and the model will not learn from it. Thus, it makes sense to limit the vocabulary size to a smaller number by passing an argument to vectorizer constructor:

In [24]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.map(lambda x: x['title']+' '+x['description']))

> **Note:** When given a vocabulary size, vectorizer selects the given number of the most frequent words. Try lowering `vocab_size` value and running the classifier training example below, and see how it affects the accuracy. You should expect some accuracy drop, but not dramatic, in lieu of higher performance.

We can now check how this works on our dataset:

In [29]:
to_bow(next(ds_train.as_numpy_iterator())['description'])

<tf.Tensor: shape=(50000,), dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>

## Training BoW classifier

Now that we have learned how to build Bag-of-Words representation of our text, let's train a classifier on top of it. First, we need to convert our dataset for training in such a way, that all positional vector representations are converted to bag-of-words representation. This can be achieved by using `map` function on top of our dataset:

In [37]:
ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(128)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(256)

Now let's define a simple classifier neural network that contains one linear layer. The size of the input vector equals to `vocab_size`, and output size corresponds to the number of classes (4). Because we are solving classification task, the final activation function is **softmax**:

In [38]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 154s 164ms/step - loss: 0.5458 - acc: 0.8765 - val_loss: 0.3679 - val_acc: 0.8961


Given that fact that we are classifying our of 4 classes, accuracy of above 80% seems like a very good result.

## Training a Classifier as One Network

Because vectorizer is also a Keras layer, we can define a network that includes it, and train it end-to-end. This will save us trouble of applying vectorization to the dataset using `map`, we can just pass the original dataset to the input of the network.

> **Note**: We would still have to apply maps to our dataset to convert fields from dictionaries (with fields `title`, `description` and `label`) to tuples. However, when loading data from disk, we can build the dataset of the required structure at the first place.

In [68]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))


Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_3 (TextVe (None, None)              0         
_________________________________________________________________
tf.one_hot_14 (TFOpLambda)   (None, None, 50000)       0         
_________________________________________________________________
tf.math.reduce_sum_14 (TFOpL (None, 50000)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 200004    
Total params: 200,004
Trainable params: 200,004
Non-trainable params: 0
_________________________________________________________________
938/938 [==============================] - 64s 68ms/step - loss: 0.5228 - acc: 0.8771 - val_loss: 0.3447 - val_acc: 0.

## BiGrams, TriGrams and N-Grams

One limitation of a bag of words approach is that some words are part of multi word expressions, for example, the word 'hot dog' has a completely different meaning than the words 'hot' and 'dog' in other contexts. If we represent words 'hot` and 'dog' always by the same vectors, it can confuse our model.

To address this, **N-gram representations** are often used in methods of document classification, where the frequency of each word, bi-word or tri-word is a useful feature for training classifiers. In bigram representation, for example, we will add all word pairs to the vocabulary, in addition to original words. 

Below is an example of how to generate a bigram bag of word representation using the Scikit Learn:


In [73]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

The main drawback of N-gram approach is that vocabulary size starts to grow extremely fast. In practice, we need to combine N-gram representation with some dimensionality reduction techniques, such as *embeddings*, which we will discuss in the next unit.

To use N-gram representation in our **AG News** dataset, we need to pass `ngrams` parameter to our `TextVectorization` constructor:

In [74]:
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(ngrams=2)
vectorizer.adapt(ds_train.map(lambda x: x['title']+' '+x['description']))
print(f"Bigram vocabulary length = {len(vectorizer.get_vocabulary())}")

Bigram vocabulary length = 1374148


> **Note:** Length of bigram vocaculary is **significantly larger**, in our case it is more than 1.3 million tokens! Thus it makes sense to limit bigram tokens as well by some reasonable number.

We could use the same code as above to train the classifier, however, it would be very memory-inefficient. In the next unit, we will train bigram classifier using embeddings. However, you can experiment with bigram classifier training in this notebook and see if you can get higher accuracy.

## Automatically calculating BoW Vectors

In the example above we have been calculating BoW vectors by hand by summing up one-hot encodings of individual words. However, latest version of TensorFlow allows to calculate BoW vectors automatically by text vectorizer, by passing `output_mode='count` parameter to the constructor. This makes defining and training our model significanly easier:

In [81]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=50000,output_mode='count'),
    keras.layers.Dense(4,input_shape=(50000,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 10s 10ms/step - loss: 0.5210 - acc: 0.8826 - val_loss: 0.3427 - val_acc: 0.9045



## Term Frequency Inverse Document Frequency TF-IDF

In BoW representation, word occurrences are evenly weighted, regardless of the word itself. However, it is clear that frequent words, such as *a*, *in*, etc. are much less important for the classification, than specialized terms. In fact, in most NLP tasks some words are more relevant than others.

**TF-IDF** stands for **term frequency–inverse document frequency**. It is a variation of bag of words, where instead of a binary 0/1 value indicating the appearance of a word in a document, a floating-point value is used, which is related to the frequency of word occurrence in the corpus.

More formally, the weight $w_{ij}$ of a word $i$ in the document $j$ is defined as:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
where
* $tf_{ij}$ is the number of occurrences of $i$ in $j$, i.e. the BoW value we have seen before
* $N$ is the number of documents in the collection
* $df_i$ is the number of documents containing the word $i$ in the whole collection

TF-IDF value $w_{ij}$ increases proportionally to the number of times a word appears in a document and is offset by the number of documents in the corpus that contains the word, which helps to adjust for the fact that some words appear more frequently than others. For example, if the word appears in *every* document in the collection, $df_i=N$, and $w_{ij}=0$, and those terms would be completely disregarded.

You can easily create TF-IDF vectorization of text using Scikit Learn:

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In Keras, `TextVectorization` layer can automatically compute TF-IDF frequencies by passing `output_mode='tf-idf'` parameter. Let's repeat the code we used above to see if using TF-IDF would increase accuracy: 

In [84]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=50000,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(50000,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training vectorizer
938/938 [==============================] - 10s 11ms/step - loss: 0.3195 - acc: 0.9034 - val_loss: 0.2563 - val_acc: 0.9186


## Conclusion 

Even though TF-IDF representations provide frequency weight to different words they are unable to represent meaning or order. As the famous linguist J. R. Firth said in 1935, “The complete meaning of a word is always contextual, and no study of meaning apart from context can be taken seriously.”. We will learn in the later units how to capture contextual information from text using language modeling.